In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split

/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-21 16:27:36.156630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

In [1]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'freeze_rate':[0, 0.2, 0.5, 0.9]
    }
}
IMAGE_SIZE=320

# Data Load

In [3]:
df = pd.read_csv('/home/lab38/Multi_proj_6/data/lettu_smallsizeimg_with_pest.csv')
df.head()

,image,grow,disease,disease-grow,area,points,original
0,/home/lab38/상추/358.jpg,1,12,12-1,NaN,NaN,NaN
1,/home/lab38/상추/235.jpg,1,12,12-1,NaN,NaN,NaN
2,/home/lab38/상추/39.jpg,0,12,12-0,NaN,NaN,NaN
3,/home/lab38/상추/345.jpg,1,12,12-1,NaN,NaN,NaN
4,/home/lab38/상추/97.jpg,1,12,12-1,NaN,NaN,NaN


In [6]:
# label encoding
disease_encoder = LabelEncoder()
disease_encoder.fit(df['disease'])
df['disease'] = disease_encoder.transform(df['disease'])
print(df['disease'].unique())

grow_encoder = LabelEncoder()
grow_encoder.fit(df['grow'])
df['grow'] = grow_encoder.transform(df['grow'])
print(df['grow'].unique())

[0 1 2]
[0 1]


In [7]:
shuffled_df = df.sample(frac=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['image'],
                                                   df['disease-grow'],
                                                   stratify=df['disease-grow'],
                                                   test_size=0.2)

In [ ]:
train_df = df[df['image'].isin(X_train)]
test_df = df[df['image'].isin(X_test)]

In [ ]:
print(train_df['disease'].unique())
print(train_df['grow'].unique())
print(test_df['disease'].unique())
print(test_df['grow'].unique())

In [11]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

# Model

In [ ]:
config_default = {
    'pretrain_net': 'inception',
    'epochs' : 1000,
    'batch_size': 20,
    'dropout' : 0.2,
    'learning_rate' : 1e-3,
    'activation': 'elu',
    'optimizer': 'adam',
    'dense': 32,
}
wandb.init(project='strawberry',
          config=config_default)
config = wandb.config

train_generator = train_gen.flow_from_dataframe(train_df, 
                                           x_col='image',
                                           y_col=['disease', 'grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)
valid_generator = valid_gen.flow_from_dataframe(test_df,
                                           x_col='image',
                                           y_col=['disease','grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)

if config.pretrain_net == 'inception' :
    MODEL_IMAGE_SIZE = 299
    base_model = inception_resnet_v2.InceptionResNetV2(
        weights='imagenet',
        include_top = False,
        input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
    )
elif config.pretrain_net == 'resnet' :
    MODEL_IMAGE_SIZE = 224
    base_model= resnet50.ResNet50(
          weights='imagenet',
          include_top=False,
          input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
         )
base_model.trainable = False 

# resizing model
input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
resizing = Model(inputs=input_data, outputs=x, name='resize')

# model
inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = resizing(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(config.dropout)(x)
backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                        name = 'diease_outputs')(backbone_out)
grow_outputs = layers.Dense(1, activation='sigmoid',
                    name = 'grow_outputs')(backbone_out)

model = Model(inputs=inputs, 
              outputs=[disease_outputs, grow_outputs],
              name='lettuce') 

if config.optimizer=='sgd':
    optimizer = SGD(learning_rate=config.learning_rate)
elif config.optimizer=='rmsprop':
    optimizer = RMSprop(learning_rate=config.learning_rate)
elif config.optimizer=='adam':
    optimizer = Adam(learning_rate=config.learning_rate)

es = EarlyStopping(monitor='loss',
                   mode='auto',
                  patience=5,
                  verbose=1)

ckpt_path = './toplayer_lettuce.ckpt'
checkpointer = ModelCheckpoint(filepath=ckpt_path,
                              monitor='val_loss',
                              save_weights_only = True,
                              save_best_only= True,
                              verbose=1)
    
model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'binary_crossentropy'
              },
              optimizer=optimizer,
              metrics=['accuracy'])

history = model.fit(train_generator,
      validation_data=valid_generator,
      verbose=1,
      epochs=config.epochs,
      callbacks=[es, checkpointer],
      steps_per_epoch=len(train_df)//config.batch_size)

# Sweep
finetuning freezing rate

In [ ]:
def finetune() :
    model.load_weights=ckpt_path
    config_default = {
        'pretrain_net': 'inception',
        'epochs' : 1000,
        'batch_size': 20,
        'dropout' : 0.2,
        'learning_rate' : 1e-3,
        'activation': 'elu',
        'optimizer': 'adam',
        'dense': 32,
        'freeze_rate' : 0.3
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    base_model.trainable=True
    fine_tune_at = int(len(base_model.layers) * config.freeze_rate)
    for layer in base_model.layers[:fine_tune_at] :
        layer.trainable = False 
        
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)  
        
    model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'binary_crossentropy'
               },
                optimizer=optimizer,
                metrics=['accuracy'])
    
    history_fine = model.fit(train_generator,
                              validation_data=valid_generator,
                              verbose=1,
                              epochs=config.epochs,
                              callbacks=[es, WandbCallbacks()],
                              steps_per_epoch=len(train_df)//config.batch_size)    

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='lettuce')

In [ ]:
wandb.agent(sweep_id, train)